# Visual GPU Log Analytics Part I: CPU Baseline in Python Pandas

Graphistry is great -- Graphistry and RAPIDS/BlazingDB is better!

This tutorial series visually analyzes Zeek/Bro network connection logs using different compute engines:

* Part I: [CPU Baseline in Python Pandas](./part_i_cpu_pandas.ipynb)
* Part II: [GPU Dataframse with RAPIDS Python cudf bindings](./part_ii_gpu_cudf)


**Part I Contents:**

Time using CPU-based Python Pandas and Graphistry for a full ETL & visual analysis flow:

1. Load data
2. Analyze data
3. Visualize data



In [1]:
#!pip install graphistry -q


import pandas as pd

import graphistry
#graphistry.register(key='MY_KEY', protocol='https', server='graphistry.site.com')
graphistry.__version__

u'0.9.63'

## 1. Load data

In [2]:
%%time
!curl https://www.secrepo.com/maccdc2012/conn.log.gz | gzip -d > conn.log
  
!head -n 3 conn.log

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  523M  100  523M    0     0   503k      0  0:17:45  0:17:45 --:--:--  483k8M    0     0   680k      0  0:13:07  0:00:44  0:12:23  486k.0M    0     0   640k      0  0:13:57  0:00:57  0:13:00  427k   0  0:14:48  0:01:17  0:13:31  425k.6M    0     0   557k      0  0:16:02  0:02:20  0:13:42  509k6M    0     0   528k      0  0:16:54  0:04:23  0:12:31  495kM    0     0   521k      0  0:17:08  0:05:37  0:11:31  498k20k      0  0:17:09  0:05:44  0:11:25  499k  0     0   519k      0  0:17:12  0:06:01  0:11:11  483k87M    0     0   519k      0  0:17:12  0:06:08  0:11:04  496k09M    0     0   514k      0  0:17:21  0:06:56  0:10:25  353k487k49M    0     0   513k      0  0:17:24  0:08:17  0:09:07  499k   0   511k      0  0:17:27  0:09:11  0:08:16  505k   0     0   508k      0  0:17:34  0:10:18  0:07:16  505k5M    0     0   507k      0  0:1

In [0]:
# OPTIONAL: For slow devices, work on a subset
#!awk 'NR % 20 == 0' < conn.log > conn-5pc.log

In [0]:
%%time
df = pd.read_csv("./conn.log", sep="\t", header=None, 
                 names=["time", "uid", "id.orig_h", "id.orig_p", "id.resp_h", "id.resp_p", "proto", "service",
                        "duration", "orig_bytes", "resp_bytes", "conn_state", "local_orig", "missed_bytes",
                        "history", "orig_pkts", "orig_ip_bytes", "resp_pkts", "resp_ip_bytes", "tunnel_parents"], 
                 na_values=['-'], index_col=False)

CPU times: user 3.87 s, sys: 107 ms, total: 3.98 s
Wall time: 3.98 s


In [0]:
df.sample(3)

,time,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents
325356,1.331909e+09,Cbmxx3GZgu3UTY1z5,192.168.202.110,54276,192.168.22.25,16632,tcp,NaN,NaN,NaN,NaN,S0,NaN,0,S,1,48,0,0,(empty)
393047,1.331919e+09,Cuefli1GJCgCJFRCWc,192.168.202.110,51711,192.168.229.156,24452,tcp,NaN,NaN,NaN,NaN,REJ,NaN,0,Sr,1,48,1,40,(empty)
1118412,1.332015e+09,CQGrBY1wejAKTunwy3,192.168.202.136,38980,192.168.21.152,3659,tcp,NaN,NaN,NaN,NaN,S0,NaN,0,S,1,44,0,0,(empty)


## 2. Analyze Data

Summarize network activities between every communicating src/dst IP,  split by connection state

In [0]:
%%time
df_summary = df\
.assign(
    sum_bytes=df.apply(lambda row: row['orig_bytes'] + row['resp_bytes'], axis=1))\
.groupby(['id.orig_h', 'id.resp_h', 'conn_state'])\
.agg({
    'time': ['min', 'max', 'size'],
    'id.resp_p':  ['nunique'],
    'uid': ['nunique'],
    'duration':   ['min', 'max', 'mean'],
    'orig_bytes': ['min', 'max', 'sum', 'mean'],
    'resp_bytes': ['min', 'max', 'sum', 'mean'],
    'sum_bytes':  ['min', 'max', 'sum', 'mean']
}).reset_index()

df_summary.columns = [' '.join(col).strip() for col in df_summary.columns.values]
df_summary = df_summary\
.rename(columns={'time size': 'count'})\
.assign(
    conn_state_uid=df_summary.apply(lambda row: row['id.orig_h'] + '_' + row['id.resp_h'] + '_' + row['conn_state'], axis=1))

CPU times: user 33.2 s, sys: 188 ms, total: 33.4 s
Wall time: 33.4 s


In [0]:
print ('# rows', len(df_summary))
df_summary.sample(3)

# rows 29784


,id.orig_h,id.resp_h,conn_state,time min,time max,count,id.resp_p nunique,uid nunique,duration min,duration max,...,orig_bytes mean,resp_bytes min,resp_bytes max,resp_bytes sum,resp_bytes mean,sum_bytes min,sum_bytes max,sum_bytes sum,sum_bytes mean,conn_state_uid
7412,192.168.202.110,192.168.27.222,S0,1.331903e+09,1.331904e+09,4,4,4,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,192.168.202.110_192.168.27.222_S0
1008,192.168.202.102,192.168.21.176,S0,1.331903e+09,1.331903e+09,1,1,1,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,192.168.202.102_192.168.21.176_S0
22046,192.168.202.79,192.168.24.98,OTH,1.331920e+09,1.331920e+09,1,1,1,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,192.168.202.79_192.168.24.98_OTH


## 3. Visualize data

* Nodes: 
  * IPs
  * Bigger when more sessions (split by connection state) involving them
* Edges:
  * src_ip -> dest_ip, split by connection state

In [0]:
%%time
hg = graphistry.hypergraph(
    df_summary,
    ['id.orig_h', 'id.resp_h'],
    direct=True,
    opts={
        'CATEGORIES': {
            'ip': ['id.orig_h', 'id.resp_h']
        }
    })

# links 29784
# events 29784
# attrib entities 3703
CPU times: user 2.27 s, sys: 42.9 ms, total: 2.31 s
Wall time: 2.31 s


In [0]:
%%time
hg['graph'].plot()

CPU times: user 3.41 s, sys: 29 ms, total: 3.44 s
Wall time: 6.42 s


## Next Steps

* Part I: [CPU Baseline in Python Pandas](./part_i_cpu_pandas.ipynb)
* Part II: [GPU Dataframe with RAPIDS Python cudf bindings](./part_ii_gpu_cudf)